In [ ]:
import tensorflow.keras as keras
import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display # If using IPython, Colab or Jupyter
import numpy as np
import tensorflow_addons as tfa
import datetime
import random
import time

In [ ]:
mnist = tf.keras.datasets.fashion_mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x = tf.random.uniform(
    (2,12,12,1),
    minval=0,
    maxval=1,
    dtype=tf.dtypes.float32,
)

h = 12
w = 12
c = 1
filter_len = 3

im = tf.reshape(tf.constant(range(2*12*12*1)), (2,12,12,1))

mask = tf.reshape(tf.range(h*w*c), (h,w,c))


i = random.randint(0,h-filter_len)
j = random.randint(0,w-filter_len)

a = tf.math.logical_and(mask / h <= j+filter_len, mask % h <= i+filter_len)
b = tf.math.logical_and(mask / w > j, mask % w > i)
d = tf.math.logical_and(a,b)

for i in range(3):
    i = random.randint(0,h-filter_len)
    j = random.randint(0,w-filter_len)

    a = tf.math.logical_and(mask / h <= j+filter_len, mask % h <= i+filter_len)
    b = tf.math.logical_and(mask / w > j, mask % w > i)
    c = tf.math.logical_and(a,b)
    d = tf.math.logical_or(d,c)


tf.where(d, tf.zeros_like(im), im)[:,:,:,0]

In [ ]:
im = tf.reshape(tf.constant(range(2*12*12*1)), (2,12,12,1))
print(im[:,:,:,0])
patches = tf.image.extract_patches(images=im,
                           sizes=[1, 4, 4, 1],
                           strides=[1, 1, 1, 1],
                           rates=[1, 1, 1, 1],
                           padding='VALID')

zeros = tf.zeros(patches.shape[-1], dtype=tf.int32)

patches * zeros


# patches = tf.extract_volume_patches(im, [2,2,2,2,1], [2,2,2,2,2], "SAME")
# def build_negative_filter(img, filter=2, masks=6):

#     patches = tf.extract_volume_patches(img, [2,2,2,2])
#     # print((n,h,w,c))
#     # i = [random.randint(0, h-filter) for _ in range(masks)]
#     # j = [random.randint(0, w-filter) for _ in range(masks)]


    


#     # def to_zero(y):
#     #     x1, x2 = y

#     #     row = int(x1/h)
#     #     col = x1 % h
#     #     # print(row, col)

#     #     for k in range(len(i)):
#     #         if row >= i[k] and row <= i[k] + filter and col >= j[k] and col <= j[k] + filter:
#     #             return 0

#     #     return 1

#     # return tf.reshape(tf.constant(list(map(to_zero, enumerate(range(n*h*w*c))))), (n,h,w,c))

# ones = tf.ones_like(im)
# twos = ones * 2
# masked = tf.math.multiply(im, build_negative_filter())
# r = masked + twos - ones
# r[:,:,:,0]

In [ ]:
class NoiseUtil:
    @staticmethod
    def filter_pixel(downsize_image_ratios = [1/16]):
        def fn(img):
            downsize_image_ratio = random.choice(downsize_image_ratios)
            resized_size_h = img.shape[1]
            resized_size_w = img.shape[2]
            
            # noisy = normalize(img) + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=(img.shape))
            # noisy = layers.MaxPooling2D(pool_size = (8, 8), padding='same')(tf.expand_dims(img, axis=0))
            down = tf.image.resize(
                img,
                [int(resized_size_h * downsize_image_ratio), int(resized_size_w * downsize_image_ratio)],
                preserve_aspect_ratio=True,
                antialias=False,
                name=None)

            up = tf.image.resize(
                down,
                [resized_size_h, resized_size_w],
                preserve_aspect_ratio=True,
                antialias=False,
                name=None)

            up = tf.cast((up), img.dtype)
            return up
        return fn

    @staticmethod
    def random_pixel_noise(img2, filter_size, patch_size, filter_fn, shape):
        
        if (len(tf.shape(img2)) < 4):
            img2 = tf.convert_to_tensor(img2[:,:,:])
            img2 = tf.expand_dims(img2, axis=0)
        else:
            img2 = tf.convert_to_tensor(img2[:,:,:,:])
  
        img2_filtered = filter_fn(img2)
        print(f"shape={shape}")
        shp = shape
        n = shp[0]
        h = shp[1]
        w = shp[2]
        c = shp[3]

        print(h)

        # mask = build_negative_filter(n, h, w, c, filter=patch_size, masks=filters)
        # x = tf.random.uniform(
        #     tf.shape(img2),
        #     minval=0,
        #     maxval=2,
        #     dtype=tf.int32,
        # )
        mask = tf.reshape(tf.range(h*w*c), (h,w,c))

        i = random.randint(0,h-patch_size)
        j = random.randint(0,w-patch_size)

        a = tf.math.logical_and(mask / h <= j+patch_size, mask % h <= i+patch_size)
        b = tf.math.logical_and(mask / w > j, mask % w > i)
        d = tf.math.logical_and(a,b)

        for i in range(filter_size):
            i = random.randint(0,h-patch_size)
            j = random.randint(0,w-patch_size)

            a = tf.math.logical_and(mask / h <= j+patch_size, mask % h <= i+patch_size)
            b = tf.math.logical_and(mask / w > j, mask % w > i)
            c = tf.math.logical_and(a,b)
            d = tf.math.logical_or(d,c)


        # im = tf.reshape(tf.constant(range(2*12*12*1)), (2,12,12,1))
        img3 = tf.where(d, tf.zeros_like(img2), img2)
        # img3 = tf.where(x == 0, img2, tf.zeros_like(img2))

        # img3 = tf.math.multiply(img2, mask)
        img4 = img2_filtered * 2
        img2 = img3 + img4 - img2_filtered


        # starts = random.sample(range(10000000), filters)
        
        # for i in range(filters):
        #     assert patch_size > 0
        #     assert patch_size % 2 == 0
        #     assert patch_size < img2.shape[1]-4
        #     assert patch_size < img2.shape[2]-4

        #     random.seed(time.time())

        #     s1 = random.randint(0,img2.shape[1]-patch_size-4)
        #     s2 = random.randint(0,img2.shape[2]-patch_size-4)
            
        #     img3 = tfa.image.cutout(img2, mask_size=[patch_size,patch_size], offset= [s1,s2])
        #     img4 = tfa.image.cutout(img2_filtered, mask_size=[patch_size,patch_size], offset= [s1,s2])

        #     img2 = img3 + img2_filtered - img4
        
        return img2

    @staticmethod
    def pixel_noise(img, filters, filter_size):
        return NoiseUtil.random_pixel_noise(img, filters, filter_size, filter_fn=NoiseUtil.filter_pixel(downsize_image_ratios = [1/4]), shape=img.shape)

In [ ]:
class ImgUtils():
    @staticmethod
    def normalize(image, as_probs=False):
        a = (tf.cast(image, tf.float32) - 127.5) if not as_probs else tf.cast(image, tf.float32)
        return a / 127.5

    @staticmethod
    def denormalize(image, cast=False):
        # return image
        img = tf.cast(image, tf.float32) * 127.5 + 127.5
        img = tf.clip_by_value(img, 0.0, 255.0)
        if cast:
            return tf.cast(img, cast)
        return img

im = np.random.normal(loc=0.0, scale=1.0, size=(1,28,28,1))
im = tf.cast(im, tf.float32) * 127.5 + 127.5
im = tf.clip_by_value(im, 0.0, 255.0)

im_processed = ImgUtils.denormalize(ImgUtils.normalize(im), cast=im.dtype)

assert np.any(im == im_processed)


class DataLoader():

    @staticmethod
    def one_from_generator(fn, shape):
        output_signature=tf.TensorSpec(shape=shape, dtype=tf.float32)
        return tf.data.Dataset.from_generator(fn, output_signature=output_signature)

    @staticmethod
    def two_from_generator(fn, shape):
        output_signature=tf.TensorSpec(shape=shape, dtype=tf.float32)
        return tf.data.Dataset.from_generator(fn, output_signature=output_signature).map(lambda x: (x,x))

class DataManager():
    def __init__(self, train_ds, test_ds = None):
        self.train_ds = train_ds
        self.test_ds = test_ds

    def set_test_samples(self, test_samples=25):
        self.test_samples = test_samples

    def set_transform(self, fn):
        self.transform = fn

    @staticmethod
    def create_label_with_input_transform(train_generator, test_generator, input_shape, transform_fn=lambda x:x, test_samples=25):
        def normalize_both(x,y):
            return ImgUtils.normalize(x), ImgUtils.normalize(y)
        
        train_ds = DataLoader.two_from_generator(train_generator, input_shape).map(normalize_both).skip(test_samples)
        test_ds = None
        if test_generator:
            test_ds = DataLoader.two_from_generator(test_generator, input_shape).map(normalize_both).take(test_samples)

        dm = DataManager(train_ds, test_ds)
        dm.set_test_samples(test_samples)
        dm.set_transform(transform_fn)

        return dm

    def get_test_data(self, batch_size):
        return self.test_ds.take(batch_size).batch(batch_size).map(self.transform)

    def get_training_data(self, batch_size):
        return self.train_ds.take(batch_size).batch(batch_size).map(self.transform)

    def print_validation(self, model=lambda x:x, batch_size=5, save=False, path="./"):
        random.seed(10)
        rows = batch_size
        cols = 2

        results = [(model(x), y) for x,y in self.get_test_data(batch_size)]
        plt.figure(figsize=(rows * 2, cols * 2))
        for x,y in results:
            assert x.shape == y.shape
            for i in range(x.shape[0]):
                im = x[i,:,:,:]
                plt.subplot(cols, rows, i+1)
                plt.imshow(ImgUtils.denormalize(im, cast=tf.uint8))
                plt.axis('off')
           

            for i in range(y.shape[0]):
                im = y[i,:,:,:]
                plt.subplot(cols, rows, i+x.shape[0]+1)
                plt.imshow(ImgUtils.denormalize(im, cast=tf.uint8))
                plt.axis('off')
        plt.subplots_adjust(wspace = 0, hspace = 0.5)
        if save:
            plt.savefig(path)
        plt.show()
           
def train_get():
   for x, y in zip(x_train, y_train):
        x = tf.expand_dims(x, axis=2)
        yield x

def test_get():
   for x in x_test:
        x = tf.expand_dims(x, axis=2)
        yield x


def add_noise(x,y):
    n = NoiseUtil.pixel_noise(x, 30, 2)
    return n,y
dm = DataManager.create_label_with_input_transform(train_get, test_get, (28,28,1), add_noise)
dm.print_validation() 

In [ ]:
print(x_test[0].shape)

In [ ]:
LATENT_SIZE = 32

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam()

def get_conv(filters):
    conv = tf.keras.Sequential()
    conv.add(tfa.layers.SpectralNormalization(
        layers.Conv2D(filters, 3, padding="same", use_bias=False)
    ))
    conv.add(layers.LeakyReLU())
    return conv
    
def get_encoder():
  encoder = tf.keras.Sequential(name="encoder")
  # encoder.add(layers.GaussianDropout(0.2))
  encoder.add(get_conv(16))
  encoder.add(layers.MaxPooling2D(pool_size = (2, 2), padding='same'))
  encoder.add(get_conv(64))
  encoder.add(layers.MaxPooling2D(pool_size = (2, 2), padding='same'))
  encoder.add(get_conv(128))
  return encoder

def get_decoder():
  decoder = tf.keras.Sequential(name="decoder")
  decoder.add(layers.UpSampling2D((2, 2), interpolation='bilinear'))
  decoder.add(get_conv(64))
  decoder.add(layers.UpSampling2D((2, 2), interpolation='bilinear'))
  decoder.add(get_conv(16))
  decoder.add(layers.Conv2D(1, 3, padding='same', activation='tanh'))
  return decoder

class AutoEncoder(tf.keras.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    
  def build(self, input_shape):
    self.encoder = get_encoder()
    self.decoder = get_decoder()
    
    self.encoder.build(input_shape=input_shape)
    
    sh = self.encoder.output_shape
    
    self.flatten = layers.Flatten()
    self.seq1 = layers.Dense(1024)
    print(sh)
    self.reshape = layers.Reshape([*sh[1:]])
    
    self.last = layers.Conv2D(1, 3, padding='same', activation='tanh')
    self.inputs_dropout = layers.Dropout(0.2)

    
  def call(self, inputs):
    x = self.inputs_dropout(inputs)
    x = self.encoder(x)
    x = self.flatten(x)
    x = self.reshape(x)
    x = self.decoder(x)
    
    # x_inputs = self.conv_input(inputs)
    
    # x = self.add([x * self.gamma, x_inputs])
    # x = self.attention(x)
    x = self.last(x)
    return x

In [ ]:
# generated_images = []

# def train_get():
#    for x, y in zip(x_train, y_train):
#         x = tf.expand_dims(x, axis=2)
#         generated_images.append(y)
#         yield x

# def test_get():
#    for x in x_test:
#         x = tf.expand_dims(x, axis=2)
#         yield x

# output_signature=tf.TensorSpec(shape=(28, 28, 1), dtype=tf.float32)

# BATCH_SIZE = 25

# def add_noise(x, filters=20, filter_size=4):
#    x= random_pixel_noise(x, filters, filter_size, filter_fn=filter_pixel(downsize_image_ratios = [1/4, 1/8]))
#    return tf.expand_dims(x, axis=-1)

# def expand(x):
#    return tf.expand_dims(x, axis=-1)

# ds_x_train = tf.data.Dataset.from_generator(train_get, output_signature=output_signature).map(lambda x: (add_noise(x),expand(x))).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ds_x_test = tf.data.Dataset.from_generator(test_get, output_signature=output_signature).map(lambda x: (add_noise(x),expand(x))).take(BATCH_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# # print(next(iter(ds_x_train))[0].shape)
# print_validation(lambda x: add_noise(x, filters=20, filter_size=4))
# print_validation(lambda x: add_noise(x, filters=20, filter_size=4))

In [ ]:


  # def validation_step(self, images):
  #   pass

  # def train_step(self, images):
  #   with tf.GradientTape() as auto_tape:
  #     generated = self.autoencoder(images)
  #     loss = cross_entropy(images, generated)
  #   gradients = auto_tape.gradient(loss, self.autoencoder.trainable_variables)
  #   opt.apply_gradients(zip(gradients, self.autoencoder.trainable_variables))

  #   return {"loss": loss}

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'logs/minst/{current_time}'
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = train_log_dir,
  write_graph=True,
  histogram_freq = 1,
  update_freq="batch"
  )

EPOCHS = 60
model = AutoEncoder()

class SkMetrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.validation_loss = []   

    def on_batch_end(self, batch, logs={}):
      originals = []
      predicted = []
      def expand_and_predict(x):
        result = model(x, training=False)
        return result
      if batch % 99 == 0 and batch > 0:
        display.clear_output()
        dm.print_validation(expand_and_predict)
        
class CustomMSE(keras.losses.Loss):
    def __init__(self, name="custom_mse"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        return mse

BATCH_SIZE = 25

# model.compile(run_eagerly=True)
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=opt, metrics=["mse"])
history = model.fit(dm.get_training_data(BATCH_SIZE).repeat(), epochs=100, steps_per_epoch=300, validation_data=dm.get_test_data(BATCH_SIZE), callbacks=[SkMetrics()])


In [ ]:
class AutoEncoder2(tf.keras.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    
  def build(self, input_shape):
    self.encoder = get_encoder()
    self.decoder = get_decoder()
    
    self.encoder.build(input_shape=input_shape)
    
    sh = self.encoder.output_shape
    
    self.flatten = layers.Flatten()
    self.seq1 = layers.Dense(1024)
    print(sh)
    self.reshape = layers.Reshape([*sh[1:]])
    
    self.last = layers.Conv2D(1, 3, padding='same', activation='sigmoid')
    self.inputs_dropout = layers.Dropout(0.2)

    
  def call(self, inputs):
    x = self.inputs_dropout(inputs)
    x = self.encoder(x)
    x = self.flatten(x)
    x = self.reshape(x)
    x = self.decoder(x)
    
    # x_inputs = self.conv_input(inputs)
    
    # x = self.add([x * self.gamma, x_inputs])
    # x = self.attention(x)
    x = self.last(x)
    return x

autoencoder2 = AutoEncoder2()
autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')

history = model.fit(dm.get_training_data().repeat(), epochs=55, steps_per_epoch=500, validation_data=dm.get_test_data(), callbacks=[SkMetrics(), tboard_callback])